# Testing

The developed classes need some thorough testing before we run the whole experiment. It would be very bad if the script breaks in the middle of tuning. This notebook lists some situations that should be tested and explains how to do so.

The tests can be performed by evaluating the objective function for a specified set of parameter values. Every class has a method <i>objfunction</i>, which takes a list of values as input.

In [ ]:
import os
os.chdir('..')
import helpers
import models
from models import OneShotCNN, OneShotTransferCNN, OneShotAutoencoder

In [ ]:
# initialize with logging and verbose
model = OneShotCNN(log=True, verbose=1)
# choose lower bound of model as example (THIS SHOULD BE DONE MANUALLY TO PERFORM THE TESTS)
x = model.xlow
print(x)

As you can see, x is just a list of numbers. This can be put in the objective function

In [ ]:
# test for given x
model.objfunction(x)

The <i>hyper_map</i> attribute of the model can be used to get the index of a hyperparameter in X. For example, the learning rate is the first entry of X (index 0).

In [ ]:
model.hyper_map['learning_rate']

to see the full list of available hyperparameters (and their order), you can print the <i>hyperparams</i> attribute:

In [ ]:
model.hyperparams

# OneShotCNN tests

For the simplest class, the OneShotCNN class, we should test the following:
- The early stopping procedure in combination with logging. Do this by training a very bad model (setting RMSProp's learning rate to 0.1 should do the trick) and have log=True on initialization.

In [ ]:
test = OneShotCNN(log=True, verbose=0)
x = test.xlow
x[test.hyper_map['learning_rate']] = 0.000000000001
test.objfunction(x)

##### Works as intended

# OneShotTransferCNN tests

For the simplest class, the OneShotCNN class, we should test the following:
- The early stopping procedure in combination with logging. Do this by training a very bad model (setting RMSProp's learning rate to 0.1 should do the trick) and have log=True on initialization.
- Having 'num_fixed_layers' larger than the total number of layers in the model.
- ..

In [ ]:
transfermodel = OneShotTransferCNN(log=True, verbose=1)

In [ ]:
x = transfermodel.xlow
# set learning rates to too very bad values
x[0] = 0.001 # big enough to learn nothing
x[1] = 0.001 # don't let it learn much in fine tuning
# set fixed layers to value bigger than the total number of layers
x[transfermodel.hyper_map['num_fixed_layers']] = 9
x[transfermodel.hyper_map['epochs']] = 1
x[transfermodel.hyper_map['finetune_epochs']] = 3

In [ ]:
transfermodel.objfunction(x)

##### Outcomes
- Early stopping works as intended and logging is not affected by it.
- The big number of fixed layers results in the classifying layer also being fixed. This way, no fine tuning occurs. <strong>Problem fixed at March 6, 2018.</strong>.

# OneShotAutoencoder tests

For the autoencoder class
- General objective evaulation
- Tuning procedure
- Whether weights are frozen and reinitialized properly
- Positioning of max pools and upsampling layers

In [ ]:
oneshotAE = OneShotAutoencoder(log=True, folds=3, verbose=1)

In [ ]:
#oneshotAE.objfunction(x)

In [ ]:
oneshotAE.tune_with_HORD(100)

##### Outcomes
- Objective evaluation works
- Tuning works
- Freezing layers works properly for the case when all layers should be frozen except the classifier. <strong>Other values need more testing</strong>
- Max pools and upsampling layers seem to position just fine.